In [2]:
import pandas as pd, numpy as np, xlsxwriter, openpyxl, os, glob, shutil
from datetime import datetime as dt
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Own defined functions

In [3]:
def get_latest_file_mtime(path, pattern='*'):
    """
    Finds the most recently MODIFIED file matching a pattern in a given directory.
    
    :param path: The directory path to search in.
    :param pattern: A glob pattern (e.g., '*.log', 'data*.csv', '*').
    :return: The full path of the latest file, or None if no files are found.
    """
    match_files = [fn for fn in os.listdir(path) if pattern in fn and not fn.startswith('~')]
    
    # 1. Create a search pattern for the files
    search_path = os.path.join(path, pattern)
    
    # 2. Get a list of all matching file paths
    list_of_files = glob.glob(search_path)
    
    # 3. Filter out directories (optional but recommended)
    files_only = [f for f in list_of_files if os.path.isfile(f) or not f.startswith('~')]

    # 4. Find the max path using the file modification time as the key
    if not files_only:
        return None
        
    latest_file = max(files_only, key=os.path.getmtime)
    
    return latest_file

def csvORexcel(path,filename,sheetname=0):
    '''
    This function is defined to read excel or csv file
    based on the file extension
    '''
    
    try:
        if file_Name.split('.')[-1].startswith('c'):
            df = pd.read_csv(path)
            return df
        elif file_Name.split('.')[-1].startswith('x'):
            try:
                df = pd.read_excel(os.path.join(path,filename), sheet_name=sheetname, engine='openpyxl',)
            except Exception:
                df = pd.read_excel(os.path.join(path,filename), sheet_name=sheetname, engine='xlrd')
            except:
                df = pd.read_csv(os.path.join(path,filename))
            return df
    except FileNotFoundError:
        print("The file name {0} has not found".format(path))


def rem_space(x):
    return ' '.join(x.split())


def readexcel(path,filename,sheetname=0):
    try:
        df = pd.read_excel(os.path.join(path,filename), sheet_name=sheetname, engine='openpyxl',)
    except:
        df = pd.read_excel(os.path.join(path,filename), sheet_name=sheetname, engine='xlrd')

    return df

# Reading Employee DB excel file

In [36]:
dPath = r'C:\Users\akash\Downloads' # Download directory path
pattern='Employee *' # Pattern to find the file

latest_file = get_latest_file_mtime(dPath, pattern,)

file_Name = latest_file.split('\\')[-1] # Extacting only file name to move the file to read folder.
print(f'File used to read = {file_Name}')

'''dest = "C:\\Python\\read\\"
if file_Name in os.listdir(dPath):
    shutil.move(os.path.join(dPath,file_Name),os.path.join(dest,file_Name))'''

active = [sn for sn in pd.ExcelFile(os.path.join(dPath, file_Name)).sheet_names if 'exit' not in sn.lower()][0] # Getting Active employees data sheet name from excel
Exit = [sn for sn in pd.ExcelFile(os.path.join(dPath, file_Name)).sheet_names if 'exit' in sn.lower()][0] # Getting Exit employees data sheet name from excel

exitEmpDB = csvORexcel(dPath, file_Name, sheetname=Exit) # Saving Exit employees data
empDB = csvORexcel(dPath, file_Name, sheetname=active) # Saving Active Emplaoyees data

# Clearing extra spaces from text columns for Exit employees data
for col in exitEmpDB.columns:
    if exitEmpDB[col].dtype == 'object':
        exitEmpDB[col].fillna(' ')
        exitEmpDB[col] = exitEmpDB[col].apply(lambda x:rem_space(str(x)))

# Clearing extra spaces from text columns for Active employees data
for col in empDB.columns:
    if empDB[col].dtype == 'object':
        print(col)
        empDB[col].fillna(' ')
        empDB[col] = empDB[col].apply(lambda x:rem_space(str(x)))

empDB.head(10)

File used to read = Employee Database-15 Dec,25.xlsx
EmployeeID
First Name
Last Name
Full Name
Title(Designation)
Level
Department
Sub Vertical
Employee status
Date of joining
Official Email ID
Employee type
Role
Project Management Role
Reporting To
Location
State


,EmployeeID,First Name,Last Name,Full Name,Title(Designation),Level,Department,Sub Vertical,Employee status,Date of joining,Official Email ID,Employee type,Role,Project Management Role,Reporting To,Location,State
0,HRM2177,MARI India,NGO,MARI India NGO,Fellow,nan,Program Delivery,Delivery,Active,2025-12-12 00:00:00,mari.ngo@haqdarshak.com,Fellow,Team member,Fellow,Jaidev Pillai HRM772,"Kurnool, Jangaon, Warangal urban, Yadadri bhuv...","Andhra Pradesh, Telangana"
1,HRM2176,Venkateswarlu,Mudavath,Venkateswarlu Mudavath,Fellow,nan,Program Delivery,Delivery,Active,2025-11-12 00:00:00,venkateswarlu.mudavathnaik@haqdarshak.com,Fellow,Team member,Fellow,S. Eswar Naik HQ00444,Nandyal,Andhra Pradesh
2,HRM2175,Soumyaranjan,Sahoo,Soumyaranjan Sahoo,Fellow,nan,Program Delivery,Delivery,Active,2025-11-12 00:00:00,soumyaranjan.sahoo@haqdarshak.com,Fellow,Team member,Fellow,Nihar Uttaray HRM1683,Athagarh,Odisha
3,HRM2174,Pathan,Mohammed Rafi,Pathan Mohammed Rafi,Fellow,nan,Program Delivery,Delivery,Active,2025-11-12 00:00:00,pathan.mohammedrafi@haqdarshak.com,Fellow,Team member,Fellow,S. Eswar Naik HQ00444,Nandyal,Andhra Pradesh
4,HRM2173,educateindiafoundation,ngo,educateindiafoundation ngo,Fellow,nan,Program Delivery,Delivery,Active,2025-09-12 00:00:00,educateindiafoundation.ngo@haqdarshak.com,Fellow,Team member,nan,Rajesh Mallik HRM850,Baragarh,Odisha
5,HRM2172,OYESTERS,NGO,OYESTERS NGO,Fellow,nan,Program Delivery,Delivery,Active,2025-08-12 00:00:00,oyesters.ngo@haqdarshak.com,Fellow,Team member,Fellow,nan,Telangana,Hyderabad
6,HRM2171,Prerna,Pawal,Prerna Pawal,Consultant,Not Applicable,Program Delivery,Delivery,Active,2025-08-12 00:00:00,pawal.prerna@haqdarshak.com,On Contract,Team member,Fellow,Surabhi Kakrania HQ00292,Pune,Maharashtra
7,HRM2170,Varsha,K,Varsha K,Fellow,nan,Program Delivery,Delivery,Active,2025-08-12 00:00:00,varsha.k@haqdarshak.com,Fellow,Team member,Fellow,Austin J HRM1428,Chennai,Tamil Nadu
8,HRM2169,Sarita,Mohapatra,Sarita Mohapatra,Fellow,nan,Programs,Monitoring & Evaluation,Active,2025-08-12 00:00:00,sarita.mohapatra@haqdarshak.com,Fellow,Team member,M and E,Radha Chauhan HRM1719,Khandagiri,Odisha
9,HRM2168,Aditya,Balaji Kamble,Aditya Balaji Kamble,Fellow,nan,Program Delivery,Delivery,Active,2025-08-12 00:00:00,adityabalaji.kamble@haqdarshak.com,Fellow,Team member,Fellow,Mayur Alandkar HRM740,Pune,Maharashtra


# Reading Delivery Hierarchy Report excel file

In [37]:
dPath = r'C:\Python\read' # Read directory path
pattern='Delivery*' # Pattern to find the file

latest_file = get_latest_file_mtime(dPath, pattern)

file_Name = latest_file.split('\\')[-1] # Extacting only file name to move the file to read folder.
print(f'File used to read = {file_Name}')

'''dest = "C:\\Python\\read\\"
if file_Name in os.listdir(dPath):
    shutil.move(os.path.join(dPath,file_Name),os.path.join(dest,file_Name))'''

df = csvORexcel(dPath, file_Name)

# Removing extra spaces from text columns in Delivery Hierarchy Report
for col in df.columns:
    if df[col].dtype == 'object':
        print(col)
        df[col].fillna(' ')
        df[col] = df[col].apply(lambda x:rem_space(str(x)))

df.head()

File used to read = Delivery_Hierarchy_Report.xlsx
PID
Status
Project Name
State
Project Role
Employee Name
Employee Email
Employee Status
Designaton
Reporting to
Reporting to Email


,PID,Status,Project Name,State,Project Role,Employee Name,Employee Email,Employee Status,Designaton,Reporting to,Reporting to Email
0,Chhattisgarh/MAHASAMUND/HES/YK/0011,Completed,CG_Mahasamund_YK,-No Value-,Delivery Head,Jaidev Pillai,jaidev.pillai@haqdarshak.com,Active,Vice President,-No Value-,-No Value-
1,Chhattisgarh/MAHASAMUND/HES/YK/0011,Completed,CG_Mahasamund_YK,-No Value-,Delivery Lead,Moonis Hasan,moonis.hasan@haqdarshak.com,Active,Manager,Jaidev Pillai,jaidev.pillai@haqdarshak.com
2,Chhattisgarh/MAHASAMUND/HES/YK/0011,Completed,CG_Mahasamund_YK,-No Value-,Lead PM,Moonis Hasan,moonis.hasan@haqdarshak.com,Active,Manager,Jaidev Pillai,jaidev.pillai@haqdarshak.com
3,Chhattisgarh/MAHASAMUND/HES/YK/0011,Completed,CG_Mahasamund_YK,Chhattisgarh,Fellow,Subir Sarkar,subir.sarkar@haqdarshak.com,Active,Fellow,Sumit Biswas,sumit.biswas@haqdarshak.com
4,Chhattisgarh/MAHASAMUND/HES/YK/0011,Completed,CG_Mahasamund_YK,Chhattisgarh,State PM,Sumit Biswas,sumit.biswas@haqdarshak.com,In - Active,Senior Coordinator,Moonis Hasan,moonis.hasan@haqdarshak.com


# Data Tranformation

In [38]:
df = df[~(df['Employee Name'].str.contains('NGO', case=True, na=False, regex=False))]

delHieRep = df[(df.Status == 'In Progress') | (df.Status == 'Open')]
delHieRep = delHieRep[(delHieRep['Employee Name'] != '-No Value-') & (delHieRep['Employee Name'] != 'Rahul Test')]
delHieRep = delHieRep[delHieRep['Project Name'] != 'Non Project Activities']
delHieRep = delHieRep[~(delHieRep['Project Name'].str.contains('Test', na=False, case=False))].copy()

for col in delHieRep.columns:
    if delHieRep[col].dtype == 'object':
        print(col)
        delHieRep[col].fillna(' ')
        delHieRep[col] = delHieRep[col].apply(lambda x:rem_space(str(x)))

delHieRep = delHieRep.merge(empDB[['Official Email ID','Employee status']], left_on='Employee Email', right_on='Official Email ID', how='left')
delHieRep.drop(columns='Official Email ID', inplace=True)
delHieRep.rename(columns={'Employee status':'Employee Status as per HR'}, inplace=True)

delHieRep['Employee Status as per HR'] = delHieRep[['Employee Status as per HR','Employee Status','Employee Email']].apply(lambda x: 'In - Active' if (x[0] in [np.NaN]) & (x[2] in exitEmpDB['Official Email ID'].value_counts().index.tolist()) else
                                                                                                                           'In - Active' if (x[0] in [np.NaN]) & (x[1] in ['In - Active','Notice Period','Terminated']) else
                                                                                                                           'Notice Period' if x[0] == 'Notice Period' else
                                                                                                                           'In - Active' if x[1] in ['In - Active','Notice Period','Terminated'] else
                                                                                                                           'Active' if (x[0] in [np.NaN]) & (x[1] == 'Active') else x[0], axis=1)

roles = delHieRep[(delHieRep['Employee Status as per HR'] != 'In - Active') & (~(delHieRep['Project Name'].str.contains('Test', na=False, case=False)))].copy()

roles = roles.merge(empDB[['Official Email ID','Sub Vertical']], left_on='Employee Email', right_on='Official Email ID', how='left')
roles.drop(columns='Official Email ID', inplace=True)

pivot1 = pd.pivot_table(roles, index=['Employee Name','Employee Email','Project Name','PID'], values='Status', aggfunc='count').reset_index()
pivot2 = pd.pivot_table(pivot1, index=['Employee Name','Employee Email'], values='PID', aggfunc='count').reset_index().sort_values(by='PID',ascending=True)
pivot2.reset_index(drop=True, inplace=True)

project_names = dict()

for email in set(pivot1['Employee Email']):
    project_names[email] = " | ".join(pivot1[pivot1['Employee Email']==email]['Project Name'].tolist())

pivot2['Project Names'] = pivot2['Employee Email'].apply(lambda x: project_names[x])
empDB = empDB.merge(pivot2[['Employee Email','PID']], left_on='Official Email ID', right_on='Employee Email', how='left')
empDB.drop(columns='Employee Email', inplace=True)
missed = empDB[(empDB.PID.isna()) & (empDB.Department=='Program Delivery')].copy()
missed = missed[~(missed['Full Name'].str.contains('NGO', case=True, na=False, regex=False))]
if len(missed) == 0:
    pivot2.sort_values(by='PID', ascending=True, inplace=True)
    pivot2.rename(columns={'PID':'Project Count'},inplace=True)
else:
    missed = missed[['Full Name','Official Email ID']]
    missed['PID']=[0]*len(missed)
    missed['Project Names']=['-']*len(missed)
    missed.rename(columns={'Full Name' : 'Employee Name',
                           'Official Email ID' : 'Employee Email'},
                  inplace=True)
    pivot2 = pd.concat([pivot2,missed], axis=0, ignore_index=True)
    pivot2.sort_values(by='PID', ascending=True, inplace=True)
    pivot2.rename(columns={'PID':'Project Count'},inplace=True)

pivot2 = pivot2.merge(empDB[['Official Email ID','Title(Designation)', 'Reporting To', 'State']], left_on='Employee Email', right_on='Official Email ID', how='left')
pivot2.rename(columns={'Title(Designation)':'Role',
                       'Reporting To':'Reporting Manager'}, inplace=True)
pivot2.drop(columns='Official Email ID', inplace=True)
pivot2 = pivot2[['Employee Name', 'Employee Email', 'Role', 'Reporting Manager', 'State', 'Project Count', 'Project Names']]

pivot2.tail(10)

PID
Status
Project Name
State
Project Role
Employee Name
Employee Email
Employee Status
Designaton
Reporting to
Reporting to Email


,Employee Name,Employee Email,Role,Reporting Manager,State,Project Count,Project Names
271,Bahelim Jivankhan,bahelim.khan@haqdarshak.com,Manager,Prerna Pawal HRM2171,Gujarat,14,All HESPL HES | Colgate Palmolive India Limite...
272,Mohd Faizan Khan,mohdf.khan@haqdarshak.com,Manager,Kajal Kiran Singh HRM931,Delhi,14,All HESPL HES | DLF Foundation Suvidha Kendra ...
273,Manju R,manju.r@haqdarshak.com,Coordinator,Prakash Singh HRM72,Karnataka,15,Aditya Birla Paints | B2C Yojana Card | Collec...
274,Meerja Basha,meerja.basha@haqdarshak.com,Senior Manager,Jaidev Pillai HRM772,Andhra Pradesh,15,"All HESPL HES | Centre for Inclusive Growth, M..."
275,Rajeev Ranjan,ranjan.rajeev@haqdarshak.com,Senior Manager,Prakash Singh HRM72,Uttarakhand,16,All HESPL HES | Annapurna Finance | Axis Max L...
276,Thakur Ravinder,thakur.ravinder@haqdarshak.com,Manager,Kajal Kiran Singh HRM931,Himachal Pradesh,18,All HESPL HES | Annapurna Finance | Bajaj Fins...
277,Prerna Pawal,pawal.prerna@haqdarshak.com,Consultant,Surabhi Kakrania HQ00292,Maharashtra,22,Bajaj Finserv- DI + Capacity Building | Centre...
278,Kanchan Deshmukh,kanchan.deshmukh@haqdarshak.com,Assistant Manager,Prerna Pawal HRM2171,Maharashtra,22,Aditya Birla Paints | All HESPL HES | Bajaj Fi...
279,Prakash Singh,prakash.singh@haqdarshak.com,Associate Vice President,Jaidev Pillai HRM772,Uttar Pradesh,28,All HESPL HES | Annapurna Finance | Axis Max L...
280,Jaidev Pillai,jaidev.pillai@haqdarshak.com,Vice President,Surabhi Kakrania HQ00292,Delhi,29,All HESPL HES | Annapurna Finance | B2C Yojana...


In [39]:
empDB.head()

,EmployeeID,First Name,Last Name,Full Name,Title(Designation),Level,Department,Sub Vertical,Employee status,Date of joining,Official Email ID,Employee type,Role,Project Management Role,Reporting To,Location,State,PID
0,HRM2177,MARI India,NGO,MARI India NGO,Fellow,nan,Program Delivery,Delivery,Active,2025-12-12 00:00:00,mari.ngo@haqdarshak.com,Fellow,Team member,Fellow,Jaidev Pillai HRM772,"Kurnool, Jangaon, Warangal urban, Yadadri bhuv...","Andhra Pradesh, Telangana",NaN
1,HRM2176,Venkateswarlu,Mudavath,Venkateswarlu Mudavath,Fellow,nan,Program Delivery,Delivery,Active,2025-11-12 00:00:00,venkateswarlu.mudavathnaik@haqdarshak.com,Fellow,Team member,Fellow,S. Eswar Naik HQ00444,Nandyal,Andhra Pradesh,NaN
2,HRM2175,Soumyaranjan,Sahoo,Soumyaranjan Sahoo,Fellow,nan,Program Delivery,Delivery,Active,2025-11-12 00:00:00,soumyaranjan.sahoo@haqdarshak.com,Fellow,Team member,Fellow,Nihar Uttaray HRM1683,Athagarh,Odisha,NaN
3,HRM2174,Pathan,Mohammed Rafi,Pathan Mohammed Rafi,Fellow,nan,Program Delivery,Delivery,Active,2025-11-12 00:00:00,pathan.mohammedrafi@haqdarshak.com,Fellow,Team member,Fellow,S. Eswar Naik HQ00444,Nandyal,Andhra Pradesh,NaN
4,HRM2173,educateindiafoundation,ngo,educateindiafoundation ngo,Fellow,nan,Program Delivery,Delivery,Active,2025-09-12 00:00:00,educateindiafoundation.ngo@haqdarshak.com,Fellow,Team member,nan,Rajesh Mallik HRM850,Baragarh,Odisha,1.0


In [43]:
pivot2.head()

,Employee Name,Employee Email,Role,Reporting Manager,State,Project Count,Project Names
0,Pathan Mohammed Rafi,pathan.mohammedrafi@haqdarshak.com,Fellow,S. Eswar Naik HQ00444,Andhra Pradesh,0,-
1,Venkateswarlu Mudavath,venkateswarlu.mudavathnaik@haqdarshak.com,Fellow,S. Eswar Naik HQ00444,Andhra Pradesh,0,-
2,Soumyaranjan Sahoo,soumyaranjan.sahoo@haqdarshak.com,Fellow,Nihar Uttaray HRM1683,Odisha,0,-
3,Dhilip Karunakaran,dhilip.karunakaran@haqdarshak.com,Senior Associate,Deepika Arora HRM1178,Tamil Nadu,1,DBS Foundation 2.0 '24
4,Firoza Biyawarwala,firoza.biyawarwala@haqdarshak.com,Fellow,Dharmisthaben Ukabhai Parmar HRM1036,Gujarat,1,Colgate Palmolive India Limited (Phase 3)


# Reading Project Master Report excel file

In [44]:
dPath = r'C:\Users\akash\Downloads' # Read directory path
pattern='Project_Master*' # Pattern to find the file

latest_file = get_latest_file_mtime(dPath, pattern)

file_Name = latest_file.split('\\')[-1] # Extacting only file name to move the file to read folder.
print(f'File used to read = {file_Name}')

'''dest = "C:\\Python\\read\\"
if file_Name in os.listdir(dPath):
    shutil.move(os.path.join(dPath,file_Name),os.path.join(dest,file_Name))'''

projectMaster = csvORexcel(dPath, file_Name)

for col in projectMaster.columns:
    if projectMaster[col].dtype == 'object':
        print(col)
        projectMaster[col].fillna(' ')
        projectMaster[col] = projectMaster[col].apply(lambda x:rem_space(str(x)))

activeProjectMaster = projectMaster[(projectMaster['Project Status'] == 'Open') | (projectMaster['Project Status'] == 'In Progress')].copy()

activeProjectMaster.head()

File used to read = Project_Master_Report.xlsx
Project Name
PID
Project Type
Multi /Single State
Lead PM
Client Name
Sales SPoC
CAM SPoC
Research SPoC
M&E SPoc
T&D SPoc
Proof Verifier Email
Project Status
Milestone Name
Date of Milestone Due Date


,Project Name,PID,Project Type,Date of Project Start Date,Date of Project End Date,Multi /Single State,Lead PM,Client Name,Sales SPoC,CAM SPoC,Research SPoC,M&E SPoc,T&D SPoc,Proof Verifier Email,Project Status,Milestone Name,Date of Milestone Due Date
82,Aditya Birla Paints,PID/ADITY2/2025/Custom/0506,Custom,2025-03-10,2025-07-31,Yes,kanchan.deshmukh@haqdarshak.com,-No Value-,ganesh.pandey_haqdarshak,kevin.noel@haqdarshak.com,ishwari.latey@haqdarshak.com,puja.kumari@haqdarshak.com,jogesh.hiriyur@haqdarshak.com,lakshmi.m@haqdarshak.com,In Progress,Advance,2025-02-28 00:00:00
83,Aditya Birla Paints,PID/ADITY2/2025/Custom/0506,Custom,2025-03-10,2025-07-31,Yes,kanchan.deshmukh@haqdarshak.com,-No Value-,ganesh.pandey_haqdarshak,kevin.noel@haqdarshak.com,ishwari.latey@haqdarshak.com,puja.kumari@haqdarshak.com,jogesh.hiriyur@haqdarshak.com,lakshmi.m@haqdarshak.com,In Progress,Milestone 1,2025-04-30 00:00:00
84,Aditya Birla Paints,PID/ADITY2/2025/Custom/0506,Custom,2025-03-10,2025-07-31,Yes,kanchan.deshmukh@haqdarshak.com,-No Value-,ganesh.pandey_haqdarshak,kevin.noel@haqdarshak.com,ishwari.latey@haqdarshak.com,puja.kumari@haqdarshak.com,jogesh.hiriyur@haqdarshak.com,lakshmi.m@haqdarshak.com,In Progress,Milestone 2,2025-06-30 00:00:00
85,Aditya Birla Paints,PID/ADITY2/2025/Custom/0506,Custom,2025-03-10,2025-07-31,Yes,kanchan.deshmukh@haqdarshak.com,-No Value-,ganesh.pandey_haqdarshak,kevin.noel@haqdarshak.com,ishwari.latey@haqdarshak.com,puja.kumari@haqdarshak.com,jogesh.hiriyur@haqdarshak.com,lakshmi.m@haqdarshak.com,In Progress,Milestone 3,2025-07-10 00:00:00
86,Aditya Birla Paints,PID/ADITY2/2025/Custom/0506,Custom,2025-03-10,2025-07-31,Yes,kanchan.deshmukh@haqdarshak.com,-No Value-,ganesh.pandey_haqdarshak,kevin.noel@haqdarshak.com,ishwari.latey@haqdarshak.com,puja.kumari@haqdarshak.com,jogesh.hiriyur@haqdarshak.com,verma.gunjan@haqdarshak.com,In Progress,Advance,2025-02-28 00:00:00


# Proof Verifier Check and Filter on Role

In [45]:
pv = activeProjectMaster['Proof Verifier Email'].value_counts().index.tolist()
pivot2['Project Names'] = pivot2[['Employee Email','Project Names']].apply(lambda x: 'Proof Verifier('+x[1]+')' if x[0] in pv else x[1], axis=1)
#pivot2.head(10)

Unallocated = pivot2[pivot2['Project Count'] == 0].copy()
Allocated = pivot2[pivot2['Project Count'] != 0].copy()
Allocated = Allocated[(Allocated['Role'] == 'Intern') | (Allocated['Role'] == 'Fellow') | (Allocated['Role'] == 'Junior Coordinator') | (Allocated['Role'] == 'Coordinator') | (Allocated['Role'] == 'Senior Coordinator') | (Allocated['Role'] == 'Associate') | (Allocated['Role'] == 'Senior Associate') | (Allocated['Role'] == 'Associate Manager') | (Allocated['Role'] == 'Assistant Manager') | (Allocated['Role'] == 'Manager')]
Allocated['Role'].value_counts()


Role
Fellow                161
Coordinator            45
Junior Coordinator     16
Intern                 12
Senior Coordinator      9
Manager                 8
Assistant Manager       6
Associate               4
Associate Manager       4
Senior Associate        3
Name: count, dtype: int64

# Adding Columns "Vertical" & "Sub Vertical"

In [46]:
Allocated = Allocated.merge(empDB[['Official Email ID','Department']], left_on='Employee Email', right_on='Official Email ID', how='left')
Allocated.drop(columns='Official Email ID', inplace=True)
Allocated = Allocated.merge(empDB[['Official Email ID','Sub Vertical']], left_on='Employee Email', right_on='Official Email ID', how='left')
Allocated.drop(columns='Official Email ID', inplace=True)
Allocated.rename(columns={'Department':'Vertical'}, inplace=True)
Allocated.head(10)

,Employee Name,Employee Email,Role,Reporting Manager,State,Project Count,Project Names,Vertical,Sub Vertical
0,Dhilip Karunakaran,dhilip.karunakaran@haqdarshak.com,Senior Associate,Deepika Arora HRM1178,Tamil Nadu,1,DBS Foundation 2.0 '24,Training & Development [Central],Training & Development [Central]
1,Firoza Biyawarwala,firoza.biyawarwala@haqdarshak.com,Fellow,Dharmisthaben Ukabhai Parmar HRM1036,Gujarat,1,Colgate Palmolive India Limited (Phase 3),Program Delivery,Delivery
2,Gada Bhargava Satya Manikanta,gadabhargava.satyamanikanta@haqdarshak.com,Associate,Deepika Arora HRM1178,Telangana,1,DBS Foundation 2.0 '24,Training & Development [Central],Training & Development [Central]
3,Gagiya Savdas Karabhai,gagiya.savdas@haqdarshak.com,Fellow,Vikesh Kokani HRM599,Gujarat,1,LTPCT Vikramgad & Ahwa (FY 2025 - 26),Program Delivery,Delivery
4,Gajendra Kumar Patel,gajendra.patel@haqdarshak.com,Fellow,Supriya Kashyap HRM478,Chhattisgarh,1,DBS Foundation 2.0 '24,Program Delivery,Delivery
5,Mahesh Babu B M,mahesh.babubm@haqdarshak.com,Intern,Manju R HRM1785,Karnataka,1,Uber Greenlight Hubs e-Shram Regs. 2025,Program Delivery,Delivery
6,Girisha K N,girisha.kn@haqdarshak.com,Fellow,Manju R HRM1785,Karnataka,1,Uber Greenlight Hubs e-Shram Regs. 2025,Program Delivery,Delivery
7,Govind Mahato,mahato.govind@haqdarshak.com,Junior Coordinator,Amol Sayaji Gaikwad HRM1817,Jharkhand,1,TATA Steel Ltd.,Program Delivery,Delivery
8,Guruprassanna Sahoo,guruprassanna.sahoo@haqdarshak.com,Associate,Deepika Arora HRM1178,Odisha,1,DBS Foundation 2.0 '24,Training & Development [Central],Training & Development [Central]
9,Dharmisthaben Ukabhai Parmar,dharmisthaben.parmar@haqdarshak.com,Coordinator,Malde Ambaliya HRM532,Gujarat,1,Colgate Palmolive India Limited (Phase 3),Program Delivery,Delivery


# Proof Verifier Details

In [47]:
pivot2[pivot2['Project Names'].str.contains('Proof Verifier', case=True, na=True, regex=False)]

,Employee Name,Employee Email,Role,Reporting Manager,State,Project Count,Project Names
220,Ganivada Appalanaidu,ganivada.appalanaidu@haqdarshak.com,Coordinator,Meerja Basha HRM1631,Andhra pradesh,3,Proof Verifier(DBS Foundation 2.0 '24 | GMR Va...


# Resource not available in EmpDB but in Delivery Hierarchy Report

In [48]:
pivot2[pivot2['Role'].isna()]

,Employee Name,Employee Email,Role,Reporting Manager,State,Project Count,Project Names


# Open Projects

In [49]:
openProjects = pd.pivot_table(roles, index=['Employee Name','Employee Email','Project Name','PID','Reporting to','State'], values='Status', aggfunc='count').reset_index()
openProjects = openProjects.merge(projectMaster[['PID','Date of Project Start Date', 'Date of Project End Date', 'Lead PM', 'Sales SPoC', 
                                                 'CAM SPoC', 'Research SPoC', 'M&E SPoc', 'T&D SPoc', 'Proof Verifier Email']], on="PID", how='left')
openProjects.rename(columns={'Date of Project Start Date':'Start Date',
                             'Date of Project End Date':'End Date'}, inplace=True)
openProjects.drop(columns='Status', inplace=True)
openProjects.head()

,Employee Name,Employee Email,Project Name,PID,Reporting to,State,Start Date,End Date,Lead PM,Sales SPoC,CAM SPoC,Research SPoC,M&E SPoc,T&D SPoc,Proof Verifier Email
0,Aanand Sagar,aanand.sagar@haqdarshak.com,DBS Foundation 2.0 '24,PID/DBSBA/2024/DI/YK/0452,Supriya Kashyap,Chhattisgarh,2024-11-01,2026-10-31,thakur.ravinder@haqdarshak.com,rasika.kaware_haqdarshak,kevin.noel@haqdarshak.com,shashank.pandey@haqdarshak.com,radha.chauhan@haqdarshak.com,humnabadkar.shreya@haqdarshak.com,-No Value-
1,Aanand Sagar,aanand.sagar@haqdarshak.com,DBS Foundation 2.0 '24,PID/DBSBA/2024/DI/YK/0452,Supriya Kashyap,Chhattisgarh,2024-11-01,2026-10-31,thakur.ravinder@haqdarshak.com,rasika.kaware_haqdarshak,kevin.noel@haqdarshak.com,shashank.pandey@haqdarshak.com,radha.chauhan@haqdarshak.com,humnabadkar.shreya@haqdarshak.com,-No Value-
2,Aanand Sagar,aanand.sagar@haqdarshak.com,DBS Foundation 2.0 '24,PID/DBSBA/2024/DI/YK/0452,Supriya Kashyap,Chhattisgarh,2024-11-01,2026-10-31,thakur.ravinder@haqdarshak.com,rasika.kaware_haqdarshak,kevin.noel@haqdarshak.com,shashank.pandey@haqdarshak.com,radha.chauhan@haqdarshak.com,humnabadkar.shreya@haqdarshak.com,-No Value-
3,Aanand Sagar,aanand.sagar@haqdarshak.com,DBS Foundation 2.0 '24,PID/DBSBA/2024/DI/YK/0452,Supriya Kashyap,Chhattisgarh,2024-11-01,2026-10-31,thakur.ravinder@haqdarshak.com,rasika.kaware_haqdarshak,kevin.noel@haqdarshak.com,shashank.pandey@haqdarshak.com,radha.chauhan@haqdarshak.com,humnabadkar.shreya@haqdarshak.com,-No Value-
4,Abhay Singh,abhay.singh@haqdarshak.com,"Centre for Inclusive Growth, Mastercard",PID/CENTR2/2024/TL/0467,Amit kumar Gupta,Uttar Pradesh,2024-09-17,2026-07-31,nupur.pant@haqdarshak.com,vedant.manore_haqdarshak,kevin.noel@haqdarshak.com,aishwarya.agarwal@haqdarshak.com,amit.sawant@haqdarshak.com,mohsin.khan@haqdarshak.com,-No Value-


# Reading User Allocation History report

In [50]:
dPath = r'C:\Users\akash\Downloads' # Read directory path
pattern='User_Allocation*' # Pattern to find the file

latest_file = get_latest_file_mtime(dPath, pattern)

file_Name = latest_file.split('\\')[-1] # Extacting only file name to move the file to read folder.
print(f'File used to read = {file_Name}')

'''dest = "C:\\Python\\read\\"
if file_Name in os.listdir(dPath):
    shutil.move(os.path.join(dPath,file_Name),os.path.join(dest,file_Name))'''

usrAllocHist = csvORexcel(dPath, file_Name)

for col in usrAllocHist.columns:
    if usrAllocHist[col].dtype == 'object':
        print(col)
        usrAllocHist[col].fillna(' ')
        usrAllocHist[col] = usrAllocHist[col].apply(lambda x:rem_space(str(x)))

usrAllocHist.head()

File used to read = User_Allocation_History_Report.xlsx
Project Name
PID
Project Status
User Name
Email
Employee Status
Role
State
End Date


,Project Name,PID,Project Status,User Name,Email,Employee Status,Role,State,Start Date,End Date
0,360 One Foundation (IIFL),PID/360ON/2023/DI/0270,Closed,Akash Bagul,akash.bagul@haqdarshak.com,In - Active,Fellow,Maharashtra,2023-06-02,2024-10-17 00:00:00
1,360 One Foundation (IIFL),PID/360ON/2023/DI/0270,Closed,Akash Bagul,akash.bagul@haqdarshak.com,In - Active,Fellow,Maharashtra,2024-10-17,2024-11-29 00:00:00
2,360 One Foundation (IIFL),PID/360ON/2023/DI/0270,Closed,Alpeshkumar Vitthalbhai Makwana,alpeshkumar.makwana@haqdarshak.com,Terminated,Fellow,-No Value-,2024-08-09,2024-10-17 00:00:00
3,360 One Foundation (IIFL),PID/360ON/2023/DI/0270,Closed,Amit Takwale,amit.takwale@haqdarshak.com,In - Active,PM,Maharashtra,2024-08-27,2024-11-29 00:00:00
4,360 One Foundation (IIFL),PID/360ON/2023/DI/0270,Closed,Amit Takwale,amit.takwale@haqdarshak.com,In - Active,Senior Coordinator,Maharashtra,2023-06-02,2024-10-17 00:00:00


In [51]:
with pd.ExcelWriter(r"F:\Haqdarshak Data\Adhoc Task\Resource Allocation Report "+str(dt.today().day)+"_"+dt.today().strftime('%b')+"'"+dt.today().strftime('%y')+".xlsx") as writer:
    Allocated.to_excel(writer, sheet_name='Allocated', index=False)
    Unallocated.to_excel(writer, sheet_name='Unallocated', index=False)
    openProjects.to_excel(writer, sheet_name='Open Projects', index=False)
    roles[['PID', 'Status', 'Project Name', 'State', 'Project Role', 'Employee Name', 'Employee Email', 'Employee Status as per HR',
           'Employee Status', 'Designaton', 'Reporting to', 'Reporting to Email', 'Sub Vertical']].to_excel(writer, sheet_name='Roles', index=False)
    usrAllocHist.to_excel(writer, sheet_name='User Allocation History Report', index=False)
                    
print('Resource Allocation Report is ready')

Resource Allocation Report is ready
